# Unificação e Limpeza de Dados da API do Recife

Este notebook realiza a extração, unificação e limpeza de dados de múltiplas APIs disponibilizadas pelo portal de dados da cidade do Recife.

## Estrutura do Código

1. **Importação de Bibliotecas**:
   - O código utiliza as bibliotecas `requests`, `pandas`, `urllib`, `re` e `ftfy`.
   - O pacote `requests` é usado para fazer requisições HTTP à API.
   - `pandas` é utilizado para manipulação de dados em formato de DataFrame.
   - `urllib` e `re` ajudam na limpeza e manipulação de strings.
   - `ftfy` é usado para corrigir problemas de codificação em strings.

2. **Definição de Resource IDs**:
   - Uma lista de `resource_ids` é criada para acessar diferentes conjuntos de dados na API.

3. **Função `fetch_data`**:
   - Esta função é responsável por buscar dados de cada `resource_id` utilizando paginação.
   - Os dados são armazenados em uma lista e convertidos em um DataFrame.

4. **Unificação de DataFrames**:
   - Os DataFrames de cada `resource_id` são concatenados em um único DataFrame.

5. **Limpeza de Dados**:
   - A função `clean_text` é utilizada para corrigir e limpar texto, removendo caracteres indesejados e corrigindo problemas de codificação.
   - Um dicionário de `correcoes` é criado para substituir palavras específicas que estão com erros.

6. **Salvamento do DataFrame**:
   - O DataFrame unificado e limpo é salvo como um arquivo Excel (`.xlsx`) na pasta padrão do Google Colab.

## Execução
- Ao executar este código, você obterá um arquivo Excel contendo os dados unificados e corrigidos, que pode ser baixado diretamente do Colab.


In [3]:
# Instalar bibliotecas necessárias (caso não estejam instaladas)
!pip install pandas requests ftfy openpyxl

import requests
import pandas as pd
import urllib.parse
import re
import ftfy  # Importando o pacote ftfy para corrigir a codificação das strings

# Lista dos 8 resource_ids (links da API)
resource_ids = [
    '6e98bbef-0eae-4729-a822-d9089598c04c',
    '97803604-a662-4d6f-b579-144ac8a3daab',
    '2ca525fa-4290-4654-9551-caedf31a512c',
    '1c41117a-679b-417d-9649-68813efd9b6b',
    '74347e0f-cc13-4319-93ee-7eb1fcf621d0',
    '82e3b1cf-44bb-477e-a447-b32c6be5694d',
    'd1c31571-0e7f-4c10-9bdb-a9e10f9e56b0',
    'ef6bbbdf-708d-40f4-a0eb-03a793d293ab'
]

# Função para carregar todos os dados de um resource_id com PAGINAÇÃO
def fetch_data(resource_id):
    base_url = f'http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id={resource_id}'
    all_records = []
    offset = 0
    limit = 1000  # Limite de registros por página
    while True:
        url = f'{base_url}&limit={limit}&offset={offset}'
        try:
            response = requests.get(url)
            response.raise_for_status()

            # Decodificar a resposta JSON corretamente
            data = response.json()

            records = data['result']['records']

            if not records:
                break  # Se não houver mais registros, interrompe a iteração

            all_records.extend(records)  # Adiciona os registros à lista total
            offset += limit  # Incrementa o offset para a próxima página

        except Exception as e:
            print(f"Erro ao carregar dados do resource_id {resource_id}: {e}")
            return None

    df = pd.DataFrame(all_records)
    return df

# Lista para armazenar os DataFrames de cada link
dataframes = []

# Iterar sobre os resource_ids e buscar os dados
for resource_id in resource_ids:
    df = fetch_data(resource_id)
    if df is not None:
        dataframes.append(df)

# Concatenar todos os DataFrames em um único
if dataframes:
    unified_df = pd.concat(dataframes, ignore_index=True)
    print("Dados unificados com sucesso!")
else:
    print("Nenhum dado foi carregado.")

# Exibir as primeiras linhas dos dados unificados
if 'unified_df' in locals():
    print(unified_df.head())

# Função para limpar e corrigir caracteres mal codificados
def clean_text(text):
    if isinstance(text, str):
        text = urllib.parse.unquote(text)  # Decodificar texto que possa estar com encoding de URL
        text = ftfy.fix_text(text)  # Corrigir caracteres mal codificados
        text = re.sub(r'[^\w\sÁÉÍÓÚáéíóúâêîôûãõç]', '', text)  # Remover caracteres não alfanuméricos, exceto acentos e cedilha
        text = re.sub(r'Ã', 'A', text)  # Substituir 'Ã' por 'A'
        text = re.sub(r'¢', 'ç', text)  # Substituir '¢' por 'ç'
        text = re.sub(r'Í', 'I', text)  # Substituir 'Í' por 'I'
        text = re.sub(r'Ç', 'C', text)  # Substituir 'Ç' por 'C'
        return text.strip()  # Remover espaços em branco extras
    return text

# Aplicar a função de limpeza na coluna 'classe'
if 'classe' in unified_df.columns:
    unified_df['classe'] = unified_df['classe'].apply(clean_text)

# Dicionário de substituições para corrigir palavras específicas
correcoes = {
    'ANSIOLATICO BENZODIAZEPANICO': 'ANSIOLÍTICO BENZODIAZEPÍNICO',
    'ANTIALÉRGICO ANTIHISTAMANICO': 'ANTIALÉRGICO ANTI-HISTAMÍNICO',
    'ANTIARRATMICO': 'ANTIARRÍTMICO',
    # Adicione mais correções conforme necessário
}

# Aplicar as substituições no DataFrame
if 'classe' in unified_df.columns:
    unified_df['classe'] = unified_df['classe'].replace(correcoes)

# Salvar o DataFrame unificado como arquivo Excel ('.xlsx')
output_file = '/content/drive/MyDrive/Colab Notebooks/Medicamentos/dados_unificados.xlsx'
unified_df.to_excel(output_file, index=False)

print(f"Dados salvos com sucesso no arquivo '{output_file}'!")


Defaulting to user installation because normal site-packages is not writeable
Erro ao carregar dados do resource_id 6e98bbef-0eae-4729-a822-d9089598c04c: 404 Client Error: Not Found for url: http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=6e98bbef-0eae-4729-a822-d9089598c04c&limit=1000&offset=0
Erro ao carregar dados do resource_id 97803604-a662-4d6f-b579-144ac8a3daab: 404 Client Error: Not Found for url: http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=97803604-a662-4d6f-b579-144ac8a3daab&limit=1000&offset=0
Erro ao carregar dados do resource_id 2ca525fa-4290-4654-9551-caedf31a512c: 404 Client Error: Not Found for url: http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=2ca525fa-4290-4654-9551-caedf31a512c&limit=1000&offset=0
Erro ao carregar dados do resource_id 74347e0f-cc13-4319-93ee-7eb1fcf621d0: 404 Client Error: Not Found for url: http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=74347e0f-cc

OSError: Cannot save file into a non-existent directory: '\content\drive\MyDrive\Colab Notebooks\Medicamentos'

#1- Etapa de "Extração":Coleta de dados de várias fontes e Carregamento e Exibição Inicial dos Dados

In [ ]:
# Etapa 1: Carregamento e Exibição Inicial dos Dados
import pandas as pd

# Definir o caminho do arquivo no Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/Medicamentos/dados_unificados.xlsx'

# Carregar o arquivo CSV unificado
df = pd.read_excel(file_path)

# Exibir as primeiras 5 linhas do DataFrame
print(df.head(10))

# Exibir as colunas disponíveis no DataFrame
# print("\nColunas disponíveis:")
print(df.columns)

# Exibir informações sobre o DataFrame (tipos de dados e valores nulos)
# print("\nInformações gerais do DataFrame:")
print(df.info())

# Exibir a quantidade de valores ausentes por coluna
# print("\nValores ausentes por coluna:")
print(df.isnull().sum())

# Exibir estatísticas descritivas para colunas numéricas
# print("\nEstatísticas descritivas para colunas numéricas:")
print(df.describe())


#2- Etapa de Transformação: Limpeza de Dados Ausentes
Nessa etapa, lidamos com valores ausentes na coluna 'cadum', removendo linhas onde este valor está ausente.

In [ ]:
# Etapa 2: Limpeza de Dados Ausentes
print("\nValores ausentes por coluna antes da limpeza:")
print(df.isnull().sum())

# Remover linhas com valores ausentes na coluna 'cadum'
df = df.dropna(subset=['cadum'])

print("\nValores ausentes por coluna após o tratamento:")
print(df.isnull().sum())
print("\nEstatísticas descritivas da coluna 'cadum' após o tratamento:")
print(df['cadum'].describe())


#3-Etapa de Transformação dos dados: Conversão de Tipos de Dados


In [ ]:
# Certifique-se de que o DataFrame 'df' está definido e contém os dados necessários

# Verificar o tipo de dados atual da coluna 'cadum'
print("\nTipos de dados atuais:")
print(df.dtypes)

# Converter 'cadum' para inteiro, lidando com valores ausentes (se necessário)
# Aqui, usamos `fillna()` para substituir NaNs por um valor antes da conversão
df['cadum'] = df['cadum'].fillna(0).astype(int)

# Converter colunas do tipo 'object' para 'string'
df['unidade'] = df['unidade'].astype('string')
df['classe'] = df['classe'].astype('string')
df['apresentacao'] = df['apresentacao'].astype('string')
df['tipo_produto'] = df['tipo_produto'].astype('string')
df['produto'] = df['produto'].astype('string')

# Verificar se a conversão foi realizada corretamente
print("\nInformações gerais após conversão:")
print(df.info())



#Etapa 4: Tratamento de Dados Duplicados


In [ ]:
# Etapa 4: Tratamento de Dados Duplicados

# Exibir o número de linhas antes de remover duplicatas
print(f"\nNúmero de linhas antes de remover duplicatas: {df.shape[0]}")

# Verificar se há registros duplicados
duplicated_rows = df[df.duplicated()]
print(f"\nNúmero de linhas duplicadas: {len(duplicated_rows)}")

# Remover duplicatas
df = df.drop_duplicates()

# Verificar se as duplicatas foram removidas
print(f"\nNúmero de linhas após remover duplicatas: {df.shape[0]}")


#Etapa 5: Tratamento de Valores Anômalos (nº negativos)

In [ ]:
# Etapa 5: Tratamento de Valores Anômalos

# Exibir estatísticas descritivas para a coluna 'quantidade'
print("\nEstatísticas descritivas da coluna 'quantidade':")
print(df['quantidade'].describe())

# Corrigir valores negativos na coluna 'quantidade'
# Substituir valores negativos por 0 (presumindo que valores negativos são erros)
df['quantidade'] = df['quantidade'].apply(lambda x: 0 if x < 0 else x)

# Exibir estatísticas descritivas da coluna 'quantidade' após correção
print("\nEstatísticas descritivas da coluna 'quantidade' após correção:")
print(df['quantidade'].describe())

# Identificar potenciais outliers
# Exemplo: definir um limite para identificar outlier


# 6- Etapa de Identificação de Outliers
```
**OPCIONAL** ◀
```

In [ ]:

# Etapa 6: Identificação de Outliers

# Calcular o IQR para a coluna 'quantidade'
Q1 = df['quantidade'].quantile(0.25)
Q3 = df['quantidade'].quantile(0.75)
IQR = Q3 - Q1

# Definir os limites inferior e superior para identificar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar outliers na coluna 'quantidade'
df['outlier_quantidade'] = df['quantidade'].apply(lambda x: 'Outlier' if x < limite_inferior or x > limite_superior else 'Normal')

# Exibir estatísticas descritivas da coluna 'quantidade' para verificação
print("\nEstatísticas descritivas da coluna 'quantidade':")
print(df['quantidade'].describe())

# Exibir a quantidade de outliers identificados
num_outliers = df['outlier_quantidade'].value_counts().get('Outlier', 0)
print(f"\nNúmero de outliers identificados: {num_outliers}")

# Exibir uma amostra dos dados com a nova coluna de identificação de outliers
print("\nAmostra dos dados com identificação de outliers:")
print(df[['quantidade', 'outlier_quantidade']].head())


**#(Não precisa executar caso faça no BI)Etapa 7 :Análise de Estoque por Unidade, Classe e Tipo de Produto (Insight)**
 -Os insights como estoque_por_unidade, qtd_total_produtosPorclasse, estoque_por_tipo_produto, estoque_por_distrito e unidades_baixo_estoque são cálculos e análises feitos com os dados, mas eles não são adicionados ao DataFrame original.

In [ ]:
# Etapa 9: Análise de Estoque por Unidade, Classe e Tipo de Produto

# Estoque total por unidade
estoque_por_unidade = df.groupby('unidade')['quantidade'].sum().reset_index() #Primeira linha: Retorna um DataFrame onde 'unidade' é uma coluna.
#estoque_por_unidade = df[["unidade","quantidade"]].groupby("unidade").sum() #Segunda linha: Retorna um DataFrame onde 'unidade' é o índice.
print("\nEstoque total por unidade:")
print(estoque_por_unidade)

# Quantidade total de produtos por classe
qtd_total_produtosPorclasse = df.groupby('classe')['quantidade'].sum().reset_index()
print("\nEstoque total por classe:")
print(qtd_total_produtosPorclasse)

# Quantidade total por tipo de produto
estoque_por_tipo_produto = df.groupby('tipo_produto')['quantidade'].sum().reset_index()
print("\nEstoque total por tipo de produto:")
print(estoque_por_tipo_produto)

#Estoque total por distrito
estoque_por_distrito = df.groupby('distrito')['quantidade'].sum().reset_index()
print("\nEstoque total por distrito:")
print(estoque_por_distrito)

#------------------INSIGHT OPCIONAIS-------------------
# Insight 1: Unidades com Baixo Estoque
# Definir um limite para estoque baixo, aqui definimos como 1000
limite_estoque_baixo = 1000
estoque_por_unidade = df.groupby('unidade')['quantidade'].sum().reset_index()

# Insight 1.1: Identificar unidades com estoque abaixo do limite
unidades_baixo_estoque = estoque_por_unidade[estoque_por_unidade['quantidade'] < limite_estoque_baixo]
print(f"\nUnidades com estoque abaixo de {limite_estoque_baixo}:")
print(unidades_baixo_estoque)

# Insight 2: Produtos com Estoque Crítico (Baixa Quantidade)
# Definir um limite de 10 unidades para identificar produtos com estoque crítico
limite_estoque_critico = 10
produtos_criticos = df[df['quantidade'] < limite_estoque_critico]
print(f"\nProdutos com estoque crítico (menos de {limite_estoque_critico} unidades):")
print(produtos_criticos[['unidade', 'produto', 'quantidade']])


#(Não precisa fazer caso faça no BI) **#Salva o arquivo Excel contendo os dados tratados e os insights derivados em único arquivo em abas separadas.**

In [ ]:
# Instalar o pacote necessário
!pip install xlsxwriter
# Etapa: Gerar colunas derivadas e salvar no arquivo principal

# Estoque total por unidade
estoque_por_unidade = df.groupby('unidade')['quantidade'].sum().reset_index()

# Quantidade total de produtos por classe
qtd_total_produtosPorclasse = df.groupby('classe')['quantidade'].sum().reset_index()

# Quantidade total por tipo de produto
estoque_por_tipo_produto = df.groupby('tipo_produto')['quantidade'].sum().reset_index()

# Estoque total por distrito
estoque_por_distrito = df.groupby('distrito')['quantidade'].sum().reset_index()

# Insight: Unidades com estoque abaixo de 1000
limite_estoque_baixo = 1000
unidades_baixo_estoque = estoque_por_unidade[estoque_por_unidade['quantidade'] < limite_estoque_baixo]

# Insight: Produtos com estoque crítico (menos de 10 unidades)
limite_estoque_critico = 10
produtos_criticos = df[df['quantidade'] < limite_estoque_critico][['unidade', 'produto', 'quantidade']]

# Salvar o arquivo com as colunas derivadas

# Importante: Se estiver utilizando no Google Colab, é necessário ter o Google Drive montado.
from google.colab import drive
drive.mount('/content/drive')

# Salvar os dados tratados em um arquivo Excel
with pd.ExcelWriter('/content/drive/MyDrive/Colab Notebooks/Medicamentos/dados_tratados_com_insight.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Dados Originais', index=False)
    estoque_por_unidade.to_excel(writer, sheet_name='Estoque por Unidade', index=False)
    qtd_total_produtosPorclasse.to_excel(writer, sheet_name='Estoque por Classe', index=False)
    estoque_por_tipo_produto.to_excel(writer, sheet_name='Estoque por Tipo Produto', index=False)
    estoque_por_distrito.to_excel(writer, sheet_name='Estoque por Distrito', index=False)
    unidades_baixo_estoque.to_excel(writer, sheet_name='Unidades Baixo Estoque', index=False)
    produtos_criticos.to_excel(writer, sheet_name='Produtos Críticos', index=False)

print("Arquivo Excel com as colunas derivadas foi salvo com sucesso.")


#(Não precisa executar caso faça no BI) 8 Etapa de Carregamento  e Construção de Colunas derivadas - salva todos os insights em um único arquivo Excel sem o dataframe original


In [ ]:
# Instalar o pacote necessário
!pip install xlsxwriter

# Etapa 9: Análise de Estoque por Unidade, Classe e Tipo de Produto

# Estoque total por unidade
estoque_por_unidade = df.groupby('unidade')['quantidade'].sum().reset_index()
print("\nEstoque total por unidade:")
print(estoque_por_unidade)

# Quantidade total de produtos por classe
qtd_total_produtosPorclasse = df.groupby('classe')['quantidade'].sum().reset_index()
print("\nEstoque total por classe:")
print(qtd_total_produtosPorclasse)

# Quantidade total por tipo de produto
estoque_por_tipo_produto = df.groupby('tipo_produto')['quantidade'].sum().reset_index()
print("\nEstoque total por tipo de produto:")
print(estoque_por_tipo_produto)

# Estoque total por distrito
estoque_por_distrito = df.groupby('distrito')['quantidade'].sum().reset_index()
print("\nEstoque total por distrito:")
print(estoque_por_distrito)

# ------------------INSIGHT OPCIONAIS-------------------
# Insight 1: Unidades com Baixo Estoque
# Definir um limite para estoque baixo, aqui definimos como 1000
limite_estoque_baixo = 1000
estoque_por_unidade = df.groupby('unidade')['quantidade'].sum().reset_index()

# Insight 1.1: Identificar unidades com estoque abaixo do limite
unidades_baixo_estoque = estoque_por_unidade[estoque_por_unidade['quantidade'] < limite_estoque_baixo]
print(f"\nUnidades com estoque abaixo de {limite_estoque_baixo}:")
print(unidades_baixo_estoque)

# Insight 2: Produtos com Estoque Crítico (Baixa Quantidade)
# Definir um limite de 10 unidades para identificar produtos com estoque crítico
limite_estoque_critico = 10
produtos_criticos = df[df['quantidade'] < limite_estoque_critico]
print(f"\nProdutos com estoque crítico (menos de {limite_estoque_critico} unidades):")
print(produtos_criticos[['unidade', 'produto', 'quantidade']])

# ------------------Salvando os Insights no Excel-------------------
import pandas as pd

# Caminho do arquivo Excel no Google Drive
output_path = '/content/drive/MyDrive/Colab Notebooks/Medicamentos/insights_estoque_medicamentos.xlsx'

# Criar um arquivo Excel com várias abas
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Aba 1: Estoque total por unidade
    estoque_por_unidade.to_excel(writer, sheet_name='Estoque por Unidade', index=False)

    # Aba 2: Quantidade total por classe
    qtd_total_produtosPorclasse.to_excel(writer, sheet_name='Estoque por Classe', index=False)

    # Aba 3: Quantidade total por tipo de produto
    estoque_por_tipo_produto.to_excel(writer, sheet_name='Estoque por Tipo Produto', index=False)

    # Aba 4: Estoque total por distrito
    estoque_por_distrito.to_excel(writer, sheet_name='Estoque por Distrito', index=False)

    # Aba 5: Unidades com estoque baixo
    unidades_baixo_estoque.to_excel(writer, sheet_name='Unidades Baixo Estoque', index=False)

    # Aba 6: Produtos com estoque crítico
    produtos_criticos[['unidade', 'produto', 'quantidade']].to_excel(writer, sheet_name='Produtos Estoque Crítico', index=False)

print(f"Arquivo Excel com os insights foi salvo em: {output_path}")


In [ ]:
#9-Etapa de Carregamento - Salvamento do dataframe

In [ ]:

# Salvar o arquivo CSV no Google Drive, usando o DataFrame 'df'
df.to_excel('/content/drive/MyDrive/Colab Notebooks/Medicamentos/dados_tratados_sem_insight.xlsx', index=False)